## Importing Packages and Source Code

In [ ]:
import json
import os
import sys
## biom used to view Qiime Artifacts
import biom

In [ ]:
## Import Source Code
from src.data import make_dataset
from src.features import build_features

In [ ]:
## Creating paths to store temp and out data
if not os.path.exists("data/temp"):
    os.makedirs("data/temp")
if not os.path.exists("data/out"):
    os.makedirs("data/out")


In [ ]:
## Used to format graph
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

## Reading Data

In [ ]:
## Obtaining file paths
with open("config/data-params.json") as fh:
    file_paths = json.load(fh)

In [ ]:
feature_table = make_dataset.read_feature_table(file_paths["feature_table_path"])
metadata = make_dataset.read_metadata(file_paths["metadata_path"])

In [ ]:
biom_table = feature_table.view(biom.Table)
print(biom_table.head())

In [ ]:
metadata.head()

# Create Features

In [ ]:
## Obtaining file paths
with open("config/feature-params.json") as fh:
    feature_params = json.load(fh)

In [ ]:
build_features.subset_metadata(metadata, **feature_params)

In [ ]:
feature_params

## Missing Data

In [ ]:
metadata['abdominal_obesity_ncep_v2'].value_counts()

In [ ]:
build_features